# 10. PyTorch Troubleshooting

> PyTorch를 사용하면서 자주 발생할 수 있는 GPU에서의 Out Of Memory (OOM) 에러 상황들의 예시를 보고 해결하는 법까지 학습합니다. 프로그래밍 도중 디버깅하기 어려운 GPU 사용시 발생할 수 있는 문제들이 발생할 때 GPU 메모리 디버깅을 도와주는 툴과 초보자가 쉽게 놓칠 수 있는 사소한 실수들의 예제들을 확인합니다.

<br>

## Reference

- [PyTorch에서 자주 발생하는 에러 질문들](https://pytorch.org/docs/stable/notes/faq.html)
- [OOM시에 GPU 메모리 flush하기](https://discuss.pytorch.org/t/how-to-clean-gpu-memory-after-a-runtimeerror/28781)
- [GPU 에러 정리](https://brstar96.github.io/shoveling/device_error_summary/)

<br>

## 10.1 OOM (Out Of Memory)

- 공포의 단어 OOM

<br>

### 10.1.1 OOM이 해결하기 어려운 이유들

- 왜 발생했는지 알기 어려움
- 어디서 발생했는지 알기 어려움
- Error backtracking 이 이상한데로 감
- 메모리의 이전상황의 파악이 어려움

<br>

## 10.2 OOM 처리 방안

- Batch Szie 줄이기 -> GPU Clean -> Run

<br>

### 10.2.1 `GPUtil` 사용하기

- nvidia-smi 처럼 GPU의 상태를 보여주는 모듈
- Colab은 환경에서 GPU 상태 보여주기 편함
- iter마다 메모리가 늘어나는지 확인!!

```python
!pip install GPUtil
```

```python
import GPUtil
GPUtil.showUtilization()
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1egHZaTfGBeRYvxfm7x9QCJ2CCVzAxwGn' width=300/>

<br>

### 10.2.2 `torch.cuda.empty_cache()`

- 사용되지 않은 GPU상 cache를 정리
- 가용 메모리를 확보
- `del` 과는 구분이 필요
- `reset` 대신 쓰기 좋은 함수
- loop 시작되기 전에 사용하는 것을 권장

```python
import torch
from GPUtil import showUtilization as gpu_usage

print("Initial GPU Usage")
gpu_usage()

tensorList = []
for x in range(10):
    tensorList.append(torch.randn(10000000,10).cuda())

print("GPU Usage after allcoating a bunch of Tensors")
gpu_usage()

del tensorList

print("GPU Usage after deleting the Tensors")
gpu_usage()

print("GPU Usage after emptying the cache")
torch.cuda.empty_cache()
gpu_usage()
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1EJdEud2BKI8lFOIdobdBJNpjINnXbxx3' width=400/>

- 출처: https://blog.paperspace.com/pytorch-memory-multi-gpu-debugging/

<br>

### 10.2.3 trainning loop에 tensor로 축적 되는 변수는 확인

- tensor로 처리된 변수는 GPU 상에 메모리 사용
- 해당 변수 loop 안에 연산에 있을 때 GPU에 computational graph를 생성(메모리 잠식)

```python
total_loss = 0
for i in range(10000):
    optimizer.zero_grad()
    output = model(input)
    loss = criterion(output) # loss 가 쌓임
    loss.backward()
    optimizer.step()
    total_loss += loss
```

- 1-d tensor의 경우 python 기본 객체로 변환하여 처리할 것

```python
total_loss = 0
for x in range(10):
    # assume loss is computed
    iter_loss = torch.randn(3,4).mean()
    iter_loss.requires_grad = True
    #total_loss += iter_loss
    total_loss += iter_loss.item() # .item() 을 붙이면 python 기본 객체로 반환된다.
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=18vXP_NunA5iEoveU0Js1UTfLQA0SytzF' width=500/>

- 출처: https://blog.paperspace.com/pytorch-memory-multi-gpu-debugging/

<br>

### 10.2.4 `del` 명령어를 적절히 사용

- 필요가 없어진 변수는 적절한 삭제가 필요함
- python의 메모리 배치 특성상 loop 이 끝나도 메모리를 차지함

```python
for x in range(10):
    i = x
print(i) # 9 is printed
```

```python
for i in range(5):
    intermediate = f(input[i])
    result += g(intermediate)
    del intermediate # 다 사용한 변수 삭제
output = h(result)
return output
```

<br>

### 10.2.5 가능 batch 사이즈 실험

- 학습시 OOM 이 발생했다면 batch 사이즈를 1로 해서 실험해보기

```python
oom = False
try:
    run_model(batch_size)
except RuntimeError: # Out of memory
    oom = True

if oom:
    for _ in range(batch_size):
        run_model(1)
```

<br>

### 10.2.6 `torch.no_grad()` 사용

- Inference 시점에서는 `torch.no_grad()` 구문을 사용
- backward pass 으로 인해 쌓이는 메모리에서 자유로움

```python
with torch.no_grad():
    for data, target in test_loader:
        output = network(data)
        test_loss += F.nll_loss(output, target, size_average=False).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
```

<br>

## 10.3 예상치 못한 에러 메세지

- OOM 말고도 유사한 에러들이 발생
  - CUDNN_STATUS_NOT_INIT
    - GPU 를 제대로 설치하지 않았을 때 발생하는 오류
  - device-side-assert
    - OOM 의 일종
- 해당 에러도 cuda와 관련하여 OOM의 일종으로 생각될 수 있으며, 적절한 코드 처리의 필요함
- 참고: [이유를 알 수 없는 GPU 에러 정리(device-side assert, CUDA error, CUDNN_STATUS_NOT_INITIALIZED 등등…)](https://brstar96.github.io/shoveling/device_error_summary/)

<br>

## 10.4 기타 주의사항

- colab에서 너무 큰 사이즈는 실행하지 말 것
  - linear, CNN, <font color='red'>LSTM</font>
- CNN의 대부분의 에러는 크기가 안 맞아서 생기는 경우
  - `torchsummary` 등으로 사이즈를 맞출 것
- tensor 의 float precision 을 16bit 로 줄일 수도 있음